In [12]:


import nibabel as nib
import nilearn as nil
from nilearn import plotting

from socket import gethostname
from yaml.loader import SafeLoader
import yaml

from level2_utils import read_yaml_for_host



print(gethostname())
# Open the file and load the file
config = read_yaml_for_host('l2_config.yml')

Benjamins-MacBook-Pro-2.local
Benjamins-MacBook-Pro-2.local


In [13]:
config.keys()

dict_keys(['rubric_dir', 'dropbox_datapath', 'nonbids_data_path', 'sst_wave1_path', 'fmriprep_dir', 'spm_l2_script_template_filepath', 'input_mat_path', 'dev_scripts_path', 'sst_level_2_path', 'spm_path', 'nii_raw_path', 'mask_location'])

In [26]:
#get the paths of the six striatal mask areas
import glob


mask_path = config['mask_location']

striatal_masks = glob.glob(mask_path + 'striatum' + '/*.nii.gz')

#load the masks and add them
striatal_joint_mask = nil.image.math_img(
    " + ".join(["img" + str(i) for i in range(len(striatal_masks))]),
    **{'img'+str(i):sm for i, sm in enumerate(striatal_masks)}
    )

#now save that mask
striatal_joint_mask_filepath = mask_path + 'aim3/striatum_joint_mask.nii.gz'
striatal_joint_mask.to_filename(striatal_joint_mask_filepath)


Next mask we want is an intersection of the striatum mask and the motor control striatal mask

In [29]:
motor_control_mask = nil.image.math_img(
    "striatum * motor_control",
    striatum = striatal_joint_mask_filepath,
    motor_control = mask_path + 'aim3/neurosynth/motor control_association-test_z_FDR_0.01.nii.gz')

motor_control_mask.to_filename(mask_path + 'aim3/motor_control_striatum_joint_mask.nii.gz')

In [ ]:
dev_scripts_abs_path = config['dev_scripts_path']
posterror_folder = 'posterror_cues_no_rt_20230512'

# roi file to open
roi_files = [dev_scripts_abs_path + 'fMRI/fx/models/SST/level2/' + posterror_folder + '/CueFollowing(CS>FS)/CueFollowing(CS>FS)striatal_cluster_1.nii',
dev_scripts_abs_path + 'fMRI/fx/models/SST/level2/' + posterror_folder + '/CueFollowing(CS>FS)/CueFollowing(CS>FS)striatal_cluster_2.nii']
# load each of the roi files
 
roi_data = []
for roi_file in roi_files:
    print(roi_file)
    ni_file = nib.load(roi_file)
    #binarize roi using nilearn
    ni_file = nil.image.math_img("img > 0", img=ni_file)
    roi_data = roi_data + [ni_file]
    #display the roi
    #plotting.plot_roi(ni_file, title=roi_file)
    print("displayed")
#combine the rois
#concatenate them
combined_roi_data = nil.image.concat_imgs(roi_data)
#then add them
combined_roi_data = nil.image.math_img("np.sum(imgs, axis=3)", imgs=combined_roi_data)

plotting.plot_roi(combined_roi_data, title="combined")
#save the combined roi
combined_roi_data.to_filename(
    dev_scripts_abs_path + '/fMRI/fx/models/SST/level2/' + posterror_folder + '/CueFollowing(CS>FS)/CueFollowing(CS>FS)striatal_cluster_combined.nii')

    


